1. Open anaconda prompt -> cd C:\Users\rs659\Desktop\AIPI 540
<!-- 2. Create a folder where you want the env files to be --> mkdir venv -->
3. conda create --prefix "C:\Users\rs659\Desktop\AIPI 540\venv" python=3.8
4. conda activate "C:\Users\rs659\Desktop\AIPI 540\venv"
5. conda install -p "C:\Users\rs659\Desktop\AIPI 540\venv" ipykernel --update-deps --force-reinstall
<!-- 6. cd Scene-Recognition -->
<!-- 7. pip install -r Requirements.txt -->
8. pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118
9. pip3 install openmim
10. git clone https://github.com/open-mmlab/mmocr.git
11. cd mmocr
12. mim install -e .

### MMOCR

In [2]:
from mmocr.apis import MMOCRInferencer
import os
import pandas as pd


# df = pd.DataFrame(columns=['Image Name','MMOcr','keras_ocr'])

# Load models into memory
# ocr = MMOCRInferencer(det='DBNet', rec='SAR')
ocr = MMOCRInferencer(det='dbnetpp_resnet50-oclip_fpnc_1200e_icdar2015', rec='abinet_20e_st-an_mj')

# Perform inference
# text = ocr('JPEG_Dataset\\6.jpg', show=False)

Loads checkpoint by http backend from path: https://download.openmmlab.com/mmocr/textdet/dbnetpp/dbnetpp_resnet50-oclip_fpnc_1200e_icdar2015/dbnetpp_resnet50-oclip_fpnc_1200e_icdar2015_20221101_124139-4ecb39ac.pth
Loads checkpoint by http backend from path: https://download.openmmlab.com/mmocr/textrecog/abinet/abinet_20e_st-an_mj/abinet_20e_st-an_mj_20221005_012617-ead8c139.pth


Downloading: "https://download.openmmlab.com/mmocr/textrecog/abinet/abinet_20e_st-an_mj/abinet_20e_st-an_mj_20221005_012617-ead8c139.pth" to C:\Users\rs659/.cache\torch\hub\checkpoints\abinet_20e_st-an_mj_20221005_012617-ead8c139.pth
c:\users\rs659\desktop\aipi 540\mmocr\mmocr\models\textrecog\module_losses\ce_module_loss.py:101: UserWarning: padding does not exist in the dictionary
  warnings.warn(
c:\users\rs659\desktop\aipi 540\mmocr\mmocr\models\textrecog\postprocessors\base.py:60: UserWarning: padding does not exist in the dictionary
  warnings.warn(


The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std



In [ ]:
text = ocr('JPEG_Dataset\\6.jpg', show=False)

In [ ]:
image_files = os.listdir('JPEG_Dataset')
ocr_text = {}

for image in image_files:
    print('JPEG_Dataset\\' + image)
    try:
        text = ocr('JPEG_Dataset\\' + image, show=False)
        ocr_text[image] = (' '.join(text['predictions'][0]['rec_texts']))
    except Exception as e:
        print(e)
        ocr_text[image] = ''
        continue

In [ ]:
df = pd.DataFrame.from_dict({'Images':list(ocr_text.keys()), 'Text-MMOcr':list(ocr_text.values())})
df.to_csv('OCRData.csv')

### KerasOCR

In [ ]:
import matplotlib.pyplot as plt

import keras_ocr
from PIL import Image
import os
import numpy as np


pipeline = keras_ocr.pipeline.Pipeline()

# images = []
# image_files = os.listdir('JPEG_Dataset')[:5]

# for image_file in image_files:
    
#     images.append(np.array(Image.open(f'JPEG_Dataset\{image_file}')))


# # Each list of predictions in prediction_groups is a list of
# # (word, box) tuples.

# # Plot the predictions
# fig, axs = plt.subplots(nrows=len(images), figsize=(20, 20))
# for ax, image, predictions in zip(axs, images, prediction_groups):
#     keras_ocr.tools.drawAnnotations(image=image, predictions=predictions, ax=ax)

In [58]:
image_files = os.listdir('JPEG_Dataset')

ocr_text_dict = {}

for image_file in image_files:
    
    prediction_groups = pipeline.recognize([np.array(Image.open(f'JPEG_Dataset\{image_file}'))])

    for prediction_group in prediction_groups:
        ocr_text = ""
        for pred in prediction_group:
            # print(pred[0])
            ocr_text += ' ' + pred[0]
        # print(ocr_text)
        ocr_text_dict[image_file] = ocr_text
    


4/4 [==============================] - 0s 23ms/step


In [61]:
# ocr_text_dict

df = pd.DataFrame.from_dict({'Images':list(ocr_text_dict.keys()), 'Text-keras_ocr':list(ocr_text_dict.values())})
df.to_csv('keras_ocrData.csv')

In [ ]:
import os
import glob

def rename_files_in_folder(folder_path):
    # Get all files in the folder
    files = glob.glob(os.path.join(folder_path, '*'))
    
    # Sort files in alphabetical order
    files.sort()
    
    # Rename files
    for i, file in enumerate(files, start=1):
        # Get the file extension
        extension = os.path.splitext(file)[1]
        
        # Construct new file path
        new_file_path = os.path.join(folder_path, f'{i}{extension}')
        
        # Rename the file
        os.rename(file, new_file_path)

In [ ]:
# Call the function with your folder path
folder = 'JPEG_Dataset'
rename_files_in_folder(folder)